# Speech-to-Text | Get transcription with speakers (OpenAI Whisper + NeMo Speaker Diarization)

- Author: [Pierre GUILLOU]()
- Full Credit: this notebook is just an update of the notebook [Whisper_Transcription_%2B_NeMo_Diarization.ipynb](https://github.com/MahmoudAshraf97/whisper-diarization/blob/main/Whisper_Transcription_%2B_NeMo_Diarization.ipynb) of [Mahmoud Ashraf](https://github.com/MahmoudAshraf97) (all texts were kept)
- Date: 07/12/2023
- Blog post: [Speech-to-Text | Get transcription WITH SPEAKERS from large audio file in any language (OpenAI Whisper + NeMo Speaker Diarization)](https://medium.com/@pierre_guillou/speech-to-text-get-transcription-with-speakers-from-large-audio-file-in-any-language-openai-8da2312f1617)

## Overview

[source: from [Mahmoud Ashraf](https://github.com/MahmoudAshraf97)] This notebook combines Whisper ASR capabilities with Voice Activity Detection (VAD) and Speaker Embedding to identify the speaker for each sentence in the transcription generated by Whisper. First, the vocals are extracted from the audio to increase the speaker embedding accuracy, then the transcription is generated using Whisper, then the timestamps are corrected and aligned using WhisperX to help minimize diarization error due to time shift. The audio is then passed into MarbleNet for VAD and segmentation to exclude silences, TitaNet is then used to extract speaker embeddings to identify the speaker for each segment, the result is then associated with the timestamps generated by WhisperX to detect the speaker for each word based on timestamps and then realigned using punctuation models to compensate for minor time shifts.

## WARNING

- This notebook runs on (free) Google Colab.
- It was tested on GPU T4.

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# check there is at least a T4 GPU
!nvidia-smi

Mon Aug 26 02:17:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Installing Dependencies

After installing the libraries, it is necessary to restart the runtime (session) in Google Colab.

In [ ]:
%%capture
!pip install git+https://github.com/m-bain/whisperX.git@a5dca2cc65b1a37f32a347e574b2c56af3a7434a
!pip install --no-build-isolation nemo_toolkit[asr]==1.21.0
!pip install git+https://github.com/facebookresearch/demucs#egg=demucs
!pip install deepmultilingualpunctuation
!pip install wget pydub
# !pip install --force-reinstall torch torchaudio torchvision
# !pip uninstall -y nvidia-cudnn-cu12
!pip install numba==0.58.0
!pip install unidecode

In [ ]:
!pip install whisperx

INFO: pip is looking at multiple versions of lightning to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 

**RESTART the runtime now!**

### Import libraries

In [ ]:
import wget

In [ ]:
import os
# import wget
from omegaconf import OmegaConf
import json
import shutil
from faster_whisper import WhisperModel
import whisperx
import torch
from pydub import AudioSegment
from nemo.collections.asr.models.msdd_models import NeuralDiarizer
from deepmultilingualpunctuation import PunctuationModel
import re
import logging
import nltk
from whisperx.alignment import DEFAULT_ALIGN_MODELS_HF, DEFAULT_ALIGN_MODELS_TORCH
from whisperx.utils import LANGUAGES, TO_LANGUAGE_CODE

import unidecode
from unidecode import unidecode
import pathlib
from pathlib import Path

[NeMo W 2024-08-26 02:26:15 transformer_bpe_models:59] Could not import NeMo NLP collection which is required for speech translation model.


### Helper Functions

In [ ]:
punct_model_langs = [
    "en",
    "fr",
    "de",
    "es",
    "it",
    "nl",
    "pt",
    "bg",
    "pl",
    "cs",
    "sk",
    "sl",
]
wav2vec2_langs = list(DEFAULT_ALIGN_MODELS_TORCH.keys()) + list(
    DEFAULT_ALIGN_MODELS_HF.keys()
)

whisper_langs = sorted(LANGUAGES.keys()) + sorted(
    [k.title() for k in TO_LANGUAGE_CODE.keys()]
)


def create_config(output_dir, DOMAIN_TYPE = "telephonic"):
    # DOMAIN_TYPE: can be meeting, telephonic, or general based on domain type of the audio file
    CONFIG_FILE_NAME = f"diar_infer_{DOMAIN_TYPE}.yaml"
    CONFIG_URL = f"https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/{CONFIG_FILE_NAME}"
    MODEL_CONFIG = os.path.join(output_dir, CONFIG_FILE_NAME)
    if not os.path.exists(MODEL_CONFIG):
        MODEL_CONFIG = wget.download(CONFIG_URL, output_dir)

    config = OmegaConf.load(MODEL_CONFIG)

    data_dir = os.path.join(output_dir, "data")
    os.makedirs(data_dir, exist_ok=True)

    meta = {
        "audio_filepath": os.path.join(output_dir, "mono_file.wav"),
        "offset": 0,
        "duration": None,
        "label": "infer",
        "text": "-",
        "rttm_filepath": None,
        "uem_filepath": None,
    }
    with open(os.path.join(data_dir, "input_manifest.json"), "w") as fp:
        json.dump(meta, fp)
        fp.write("\n")

    pretrained_vad = "vad_multilingual_marblenet"
    pretrained_speaker_model = "titanet_large"
    config.num_workers = 0  # Workaround for multiprocessing hanging with ipython issue
    config.diarizer.manifest_filepath = os.path.join(data_dir, "input_manifest.json")
    config.diarizer.out_dir = (
        output_dir  # Directory to store intermediate files and prediction outputs
    )

    config.diarizer.speaker_embeddings.model_path = pretrained_speaker_model
    config.diarizer.oracle_vad = (
        False  # compute VAD provided with model_path to vad config
    )
    config.diarizer.clustering.parameters.oracle_num_speakers = False

    # Here, we use our in-house pretrained NeMo VAD model
    config.diarizer.vad.model_path = pretrained_vad
    config.diarizer.vad.parameters.onset = 0.8
    config.diarizer.vad.parameters.offset = 0.6
    config.diarizer.vad.parameters.pad_offset = -0.05
    config.diarizer.msdd_model.model_path = (
        "diar_msdd_telephonic"  # Telephonic speaker diarization model
    )

    return config


def get_word_ts_anchor(s, e, option="start"):
    if option == "end":
        return e
    elif option == "mid":
        return (s + e) / 2
    return s


def get_words_speaker_mapping(wrd_ts, spk_ts, word_anchor_option="start"):
    s, e, sp = spk_ts[0]
    wrd_pos, turn_idx = 0, 0
    wrd_spk_mapping = []
    for wrd_dict in wrd_ts:
        ws, we, wrd = (
            int(wrd_dict["start"] * 1000),
            int(wrd_dict["end"] * 1000),
            wrd_dict["word"],
        )
        wrd_pos = get_word_ts_anchor(ws, we, word_anchor_option)
        while wrd_pos > float(e):
            turn_idx += 1
            turn_idx = min(turn_idx, len(spk_ts) - 1)
            s, e, sp = spk_ts[turn_idx]
            if turn_idx == len(spk_ts) - 1:
                e = get_word_ts_anchor(ws, we, option="end")
        wrd_spk_mapping.append(
            {"word": wrd, "start_time": ws, "end_time": we, "speaker": sp}
        )
    return wrd_spk_mapping


sentence_ending_punctuations = ".?!"


def get_first_word_idx_of_sentence(word_idx, word_list, speaker_list, max_words):
    is_word_sentence_end = (
        lambda x: x >= 0 and word_list[x][-1] in sentence_ending_punctuations
    )
    left_idx = word_idx
    while (
        left_idx > 0
        and word_idx - left_idx < max_words
        and speaker_list[left_idx - 1] == speaker_list[left_idx]
        and not is_word_sentence_end(left_idx - 1)
    ):
        left_idx -= 1

    return left_idx if left_idx == 0 or is_word_sentence_end(left_idx - 1) else -1


def get_last_word_idx_of_sentence(word_idx, word_list, max_words):
    is_word_sentence_end = (
        lambda x: x >= 0 and word_list[x][-1] in sentence_ending_punctuations
    )
    right_idx = word_idx
    while (
        right_idx < len(word_list)
        and right_idx - word_idx < max_words
        and not is_word_sentence_end(right_idx)
    ):
        right_idx += 1

    return (
        right_idx
        if right_idx == len(word_list) - 1 or is_word_sentence_end(right_idx)
        else -1
    )


def get_realigned_ws_mapping_with_punctuation(
    word_speaker_mapping, max_words_in_sentence=50
):
    is_word_sentence_end = (
        lambda x: x >= 0
        and word_speaker_mapping[x]["word"][-1] in sentence_ending_punctuations
    )
    wsp_len = len(word_speaker_mapping)

    words_list, speaker_list = [], []
    for k, line_dict in enumerate(word_speaker_mapping):
        word, speaker = line_dict["word"], line_dict["speaker"]
        words_list.append(word)
        speaker_list.append(speaker)

    k = 0
    while k < len(word_speaker_mapping):
        line_dict = word_speaker_mapping[k]
        if (
            k < wsp_len - 1
            and speaker_list[k] != speaker_list[k + 1]
            and not is_word_sentence_end(k)
        ):
            left_idx = get_first_word_idx_of_sentence(
                k, words_list, speaker_list, max_words_in_sentence
            )
            right_idx = (
                get_last_word_idx_of_sentence(
                    k, words_list, max_words_in_sentence - k + left_idx - 1
                )
                if left_idx > -1
                else -1
            )
            if min(left_idx, right_idx) == -1:
                k += 1
                continue

            spk_labels = speaker_list[left_idx : right_idx + 1]
            mod_speaker = max(set(spk_labels), key=spk_labels.count)
            if spk_labels.count(mod_speaker) < len(spk_labels) // 2:
                k += 1
                continue

            speaker_list[left_idx : right_idx + 1] = [mod_speaker] * (
                right_idx - left_idx + 1
            )
            k = right_idx

        k += 1

    k, realigned_list = 0, []
    while k < len(word_speaker_mapping):
        line_dict = word_speaker_mapping[k].copy()
        line_dict["speaker"] = speaker_list[k]
        realigned_list.append(line_dict)
        k += 1

    return realigned_list


def get_sentences_speaker_mapping(word_speaker_mapping, spk_ts):
    sentence_checker = nltk.tokenize.PunktSentenceTokenizer().text_contains_sentbreak
    s, e, spk = spk_ts[0]
    prev_spk = spk

    snts = []
    snt = {"speaker": f"Speaker {spk}", "start_time": s, "end_time": e, "text": ""}

    for wrd_dict in word_speaker_mapping:
        wrd, spk = wrd_dict["word"], wrd_dict["speaker"]
        s, e = wrd_dict["start_time"], wrd_dict["end_time"]
        if spk != prev_spk or sentence_checker(snt["text"] + " " + wrd):
            snts.append(snt)
            snt = {
                "speaker": f"Speaker {spk}",
                "start_time": s,
                "end_time": e,
                "text": "",
            }
        else:
            snt["end_time"] = e
        snt["text"] += wrd + " "
        prev_spk = spk

    snts.append(snt)
    return snts


def get_speaker_aware_transcript(sentences_speaker_mapping, f):
    previous_speaker = sentences_speaker_mapping[0]["speaker"]
    f.write(f"{previous_speaker}: ")

    for sentence_dict in sentences_speaker_mapping:
        speaker = sentence_dict["speaker"]
        sentence = sentence_dict["text"]

        # If this speaker doesn't match the previous one, start a new paragraph
        if speaker != previous_speaker:
            f.write(f"\n\n{speaker}: ")
            previous_speaker = speaker

        # No matter what, write the current sentence
        f.write(sentence + " ")


def format_timestamp(
    milliseconds: float, always_include_hours: bool = False, decimal_marker: str = "."
):
    assert milliseconds >= 0, "non-negative timestamp expected"

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
    return (
        f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"
    )


def write_srt(transcript, file):
    """
    Write a transcript to a file in SRT format.

    """
    for i, segment in enumerate(transcript, start=1):
        # write srt lines
        print(
            f"{i}\n"
            f"{format_timestamp(segment['start_time'], always_include_hours=True, decimal_marker=',')} --> "
            f"{format_timestamp(segment['end_time'], always_include_hours=True, decimal_marker=',')}\n"
            f"{segment['speaker']}: {segment['text'].strip().replace('-->', '->')}\n",
            file=file,
            flush=True,
        )


def find_numeral_symbol_tokens(tokenizer):
    numeral_symbol_tokens = [
        -1,
    ]
    for token, token_id in tokenizer.get_vocab().items():
        has_numeral_symbol = any(c in "0123456789%$£" for c in token)
        if has_numeral_symbol:
            numeral_symbol_tokens.append(token_id)
    return numeral_symbol_tokens


def _get_next_start_timestamp(word_timestamps, current_word_index):
    # if current word is the last word
    if current_word_index == len(word_timestamps) - 1:
        return word_timestamps[current_word_index]["start"]

    next_word_index = current_word_index + 1
    while current_word_index < len(word_timestamps) - 1:
        if word_timestamps[next_word_index].get("start") is None:
            # if next word doesn't have a start timestamp
            # merge it with the current word and delete it
            word_timestamps[current_word_index]["word"] += (
                " " + word_timestamps[next_word_index]["word"]
            )

            word_timestamps[next_word_index]["word"] = None
            next_word_index += 1

        else:
            return word_timestamps[next_word_index]["start"]


def filter_missing_timestamps(word_timestamps):
    # handle the first and last word
    if word_timestamps[0].get("start") is None:
        word_timestamps[0]["start"] = 0
        word_timestamps[0]["end"] = _get_next_start_timestamp(word_timestamps, 0)

    result = [
        word_timestamps[0],
    ]

    for i, ws in enumerate(word_timestamps[1:], start=1):
        # if ws doesn't have a start and end
        # use the previous end as start and next start as end
        if ws.get("start") is None and ws.get("word") is not None:
            ws["start"] = word_timestamps[i - 1]["end"]
            ws["end"] = _get_next_start_timestamp(word_timestamps, i)

        if ws["word"] is not None:
            result.append(ws)
    return result


def cleanup(path: str):
    """path could either be relative or absolute."""
    # check if file or directory exists
    if os.path.isfile(path) or os.path.islink(path):
        # remove file
        os.remove(path)
    elif os.path.isdir(path):
        # remove directory and all its content
        shutil.rmtree(path)
    else:
        raise ValueError("Path {} is not a file or dir.".format(path))


def process_language_arg(language: str, model_name: str):
    """
    Process the language argument to make sure it's valid and convert language names to language codes.
    """
    if language is not None:
        language = language.lower()
    if language not in LANGUAGES:
        if language in TO_LANGUAGE_CODE:
            language = TO_LANGUAGE_CODE[language]
        else:
            raise ValueError(f"Unsupported language: {language}")

    if model_name.endswith(".en") and language != "en":
        if language is not None:
            logging.warning(
                f"{model_name} is an English-only model but received '{language}'; using English instead."
            )
        language = "en"
    return language


def transcribe(
    audio_file: str,
    language: str,
    model_name: str,
    compute_dtype: str,
    suppress_numerals: bool,
    device: str,
):
    from faster_whisper import WhisperModel
    from helpers import find_numeral_symbol_tokens, wav2vec2_langs

    # Faster Whisper non-batched
    # Run on GPU with FP16
    whisper_model = WhisperModel(model_name, device=device, compute_type=compute_dtype)

    # or run on GPU with INT8
    # model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
    # or run on CPU with INT8
    # model = WhisperModel(model_size, device="cpu", compute_type="int8")

    if suppress_numerals:
        numeral_symbol_tokens = find_numeral_symbol_tokens(whisper_model.hf_tokenizer)
    else:
        numeral_symbol_tokens = None

    if language is not None and language in wav2vec2_langs:
        word_timestamps = False
    else:
        word_timestamps = True

    segments, info = whisper_model.transcribe(
        audio_file,
        language=language,
        beam_size=5,
        word_timestamps=word_timestamps,  # TODO: disable this if the language is supported by wav2vec2
        suppress_tokens=numeral_symbol_tokens,
        vad_filter=True,
    )
    whisper_results = []
    for segment in segments:
        whisper_results.append(segment._asdict())
    # clear gpu vram
    del whisper_model
    torch.cuda.empty_cache()
    return whisper_results, language


def transcribe_batched(
    audio_file: str,
    language: str,
    batch_size: int,
    model_name: str,
    compute_dtype: str,
    suppress_numerals: bool,
    device: str,
):
    import whisperx

    # Faster Whisper batched
    whisper_model = whisperx.load_model(
        model_name,
        device,
        compute_type=compute_dtype,
        asr_options={"suppress_numerals": suppress_numerals},
    )
    audio = whisperx.load_audio(audio_file)
    result = whisper_model.transcribe(audio, language=language, batch_size=batch_size)
    del whisper_model
    torch.cuda.empty_cache()
    return result["segments"], result["language"]

In [ ]:
# no space, punctuation, accent in lower string
def cleanString(string):
    cleanString = unidecode(string)
    # cleanString = re.sub('\W+','_', cleanString)
    cleanString = re.sub(r'[^\w\s]','',cleanString)
    cleanString = cleanString.replace(" ", "_")
    return cleanString.lower()

# rename audio filename to get name without accent, no space, in lower case
def rename_file(filepath):
    suffix = Path(filepath).suffix
    if str(Path(filepath).parent) != ".":
        new_filepath = str(Path(filepath).parent) + cleanString(filepath.replace(suffix, "")) + suffix
    else:
        new_filepath = cleanString(filepath.replace(suffix, "")) + suffix
    os.rename(filepath, new_filepath)
    return new_filepath

### Setup options

In [ ]:
# Name of the audio file
audio_path = "00bcae999c09402cb74a20549a188040_20230429t15_03_utc.wav"
# audio_path = "audio1.wav"
# audio_path = "audio2.wav"

# rename audio filename if necessary to get string without accent, space, in lower case
audio_path = rename_file(audio_path)

FileNotFoundError: [Errno 2] No such file or directory: '00bcae999c09402cb74a20549a188040_20230429t15_03_utc.wav' -> '00bcae999c09402cb74a20549a188040_20230429t15_03_utc.wav'

In [ ]:
# (Option) Whether to enable music removal from speech, helps increase diarization quality but uses alot of ram
enable_stemming = False

# (choose from 'tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large-v1', 'large-v2', 'large-v3', 'large')
# whisper_model_name = "large-v2"
whisper_model_name = "large-v3"

# replaces numerical digits with their pronounciation, increases diarization accuracy
suppress_numerals = True

batch_size = 8

language = None  # autodetect language

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# check device
device


'cuda'

# Processing

## (Option) Separating music from speech using Demucs

---

By isolating the vocals from the rest of the audio, it becomes easier to identify and track individual speakers based on the spectral and temporal characteristics of their speech signals. Source separation is just one of many techniques that can be used as a preprocessing step to help improve the accuracy and reliability of the overall diarization process.

In [ ]:
%%time
if enable_stemming:
    # Isolate vocals from the rest of the audio

    return_code = os.system(
        f'python3 -m demucs.separate -n htdemucs --two-stems=vocals "{audio_path}" -o "temp_outputs"'
    )

    if return_code != 0:
        logging.warning("Source splitting failed, using original audio file.")
        vocal_target = audio_path
    else:
        vocal_target = os.path.join(
            "temp_outputs",
            "htdemucs",
            os.path.splitext(os.path.basename(audio_path))[0],
            "vocals.wav",
        )
else:
    vocal_target = audio_path

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 8.82 µs


## Transcriping audio using Whisper and realligning timestamps using Wav2Vec2
---
This code uses two different open-source models to transcribe speech and perform forced alignment on the resulting transcription.

The first model is called OpenAI Whisper, which is a speech recognition model that can transcribe speech with high accuracy. The code loads the whisper model and uses it to transcribe the vocal_target file.

The output of the transcription process is a set of text segments with corresponding timestamps indicating when each segment was spoken.


In [ ]:
%%time
if device == "cuda": compute_type = "float16"
# or run on GPU with INT8
# compute_type = "int8_float16"
# or run on CPU with INT8
else: compute_type = "int8"

if batch_size != 0:
    whisper_results, language = transcribe_batched(
        vocal_target,
        language,
        batch_size,
        whisper_model_name,
        compute_type,
        suppress_numerals,
        device,
    )
else:
    whisper_results, language = transcribe(
        vocal_target,
        language,
        whisper_model_name,
        compute_type,
        suppress_numerals,
        device,
    )

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 10.3MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.7. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.
Detected language: es (0.99) in first 30s of audio...
Suppressing numeral and symbol tokens: [3, 4, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 502, 568, 628, 805, 945, 1017, 1025, 1266, 1294, 1360, 1386, 1525, 1614, 1649, 1722, 1848, 1958, 2009, 2119, 2217, 2272, 2319, 2331, 2443, 2625, 2803, 2975, 3165, 3279, 3282, 3356, 3405, 3446, 3499, 3552, 3705, 4022, 4060, 4289, 4303, 4436, 4550, 4688, 4702, 4762, 4808, 5080, 5211, 5254, 5285, 5348, 5853, 5867, 5923, 6071, 6074, 6096, 6375, 6494, 6549, 6591, 6641, 6673, 6856, 6866, 6879, 6905, 6976, 7143, 7201, 7271, 7490, 7526, 7546, 7551, 7560, 7562, 7629, 7634, 7668, 7771, 7773, 7911, 7998, 8132, 8227, 8423, 8451, 8465, 8494, 8652, 8794, 8858, 8923, 9012, 9125, 9356, 9413, 9562, 9657, 9714, 9754, 10076,

## Aligning the transcription with the original audio using Wav2Vec2
---
The second model used is called wav2vec2, which is a large-scale neural network that is designed to learn representations of speech that are useful for a variety of speech processing tasks, including speech recognition and alignment.

The code loads the wav2vec2 alignment model and uses it to align the transcription segments with the original audio signal contained in the vocal_target file. This process involves finding the exact timestamps in the audio signal where each segment was spoken and aligning the text accordingly.

By combining the outputs of the two models, the code produces a fully aligned transcription of the speech contained in the vocal_target file. This aligned transcription can be useful for a variety of speech processing tasks, such as speaker diarization, sentiment analysis, and language identification.

If there's no Wav2Vec2 model available for your language, word timestamps generated by whisper will be used instead.

In [ ]:
%%time
if language in wav2vec2_langs:
    device = "cuda"
    alignment_model, metadata = whisperx.load_align_model(
        language_code=language, device=device
    )
    result_aligned = whisperx.align(
        whisper_results, alignment_model, metadata, vocal_target, device
    )
    word_timestamps = filter_missing_timestamps(result_aligned["word_segments"])

    # clear gpu vram
    del alignment_model
    torch.cuda.empty_cache()
else:
    assert batch_size == 0, (  # TODO: add a better check for word timestamps existence
        f"Unsupported language: {language}, use --batch_size to 0"
        " to generate word timestamps using whisper directly and fix this error."
    )
    word_timestamps = []
    for segment in whisper_results:
        for word in segment["words"]:
            word_timestamps.append({"word": word[2], "start": word[0], "end": word[1]})

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_voxpopuli_base_10k_asr_es.pt" to /root/.cache/torch/hub/checkpoints/wav2vec2_voxpopuli_base_10k_asr_es.pt
100%|██████████| 360M/360M [00:02<00:00, 143MB/s]


CPU times: user 8.28 s, sys: 1.81 s, total: 10.1 s
Wall time: 11.9 s


## Convert audio to mono for NeMo combatibility

In [ ]:
%%time
sound = AudioSegment.from_file(vocal_target).set_channels(1)
ROOT = os.getcwd()
temp_path = os.path.join(ROOT, "temp_outputs")
os.makedirs(temp_path, exist_ok=True)
sound.export(os.path.join(temp_path, "mono_file.wav"), format="wav")

CPU times: user 33.8 ms, sys: 15.6 ms, total: 49.4 ms
Wall time: 96.7 ms


<_io.BufferedRandom name='/content/temp_outputs/mono_file.wav'>

## Speaker Diarization using NeMo MSDD Model
---
This code uses a model called Nvidia NeMo MSDD (Multi-scale Diarization Decoder) to perform speaker diarization on an audio signal. Speaker diarization is the process of separating an audio signal into different segments based on who is speaking at any given time.

In [ ]:
%%time
# Initialize NeMo MSDD diarization model
# DOMAIN_TYPE: can be meeting, telephonic, or general based on domain type of the audio file
msdd_model = NeuralDiarizer(cfg=create_config(temp_path, DOMAIN_TYPE="telephonic")).to("cuda")
msdd_model.diarize()

del msdd_model
torch.cuda.empty_cache()

[NeMo I 2024-08-17 16:33:38 msdd_models:1092] Loading pretrained diar_msdd_telephonic model from NGC
[NeMo I 2024-08-17 16:33:38 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/diar_msdd_telephonic/versions/1.0.1/files/diar_msdd_telephonic.nemo to /root/.cache/torch/NeMo/NeMo_1.21.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo
[NeMo I 2024-08-17 16:33:39 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2024-08-17 16:33:41 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: true
    
[NeMo W 2024-08-17 16:33:41 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: false
    
[NeMo W 2024-08-17 16:33:41 modelPT:174] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2024-08-17 16:33:41 features:289] PADDING: 16
[NeMo I 2024-08-17 16:33:41 features:289] PADDING: 16
[NeMo I 2024-08-17 16:33:43 save_restore_connector:249] Model EncDecDiarLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2024-08-17 16:33:43 features:289] PADDING: 16
[NeMo I 2024-08-17 16:33:44 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2024-08-17 16:33:44 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/vad_multilingual_marblenet/versions/1.10.0/files/vad_multilingual_marblenet.nemo to /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2024-08-17 16:33:44 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2024-08-17 16:33:45 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2024-08-17 16:33:45 features:289] PADDING: 16
[NeMo I 2024-08-17 16:33:45 save_restore_connector:249] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2024-08-17 16:33:45 msdd_models:864] Multiscale Weights: [1, 1, 1, 1, 1]
[NeMo I 2024-08-17 16:33:45 msdd_models:865] Clustering Parameters: {
        "oracle_num_speakers": false,
        "max_num_speakers": 8,
        "enhanced_count_thres": 80,
        "max_rp_threshold": 0.25,
        "sparse_search_volume": 30,
        "maj_vote_spk_count": false,
        "chunk_cluster_count": 50,
        "embeddings_per_chunk": 10000
    }
[NeMo I 2024-08-17 16:33:45 speaker_utils:93] Number of files to diarize: 1
[NeMo I 2024-08-17 16:33:45 clustering_diarizer:309] Split long audio file to avoid CUDA memory issue


...

[NeMo I 2024-08-17 16:34:20 speaker_utils:93] Number of files to diarize: 1


[NeMo W 2024-08-17 16:34:20 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2024-08-17 16:34:20 speaker_utils:93] Number of files to diarize: 1


[NeMo W 2024-08-17 16:34:20 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2024-08-17 16:34:20 msdd_models:1431]   
    
CPU times: user 36.5 s, sys: 1.48 s, total: 38 s
Wall time: 42.1 s


## Mapping Spekers to Sentences According to Timestamps

In [ ]:
%%time
# Reading timestamps <> Speaker Labels mapping

speaker_ts = []
with open(os.path.join(temp_path, "pred_rttms", "mono_file.rttm"), "r") as f:
    lines = f.readlines()
    for line in lines:
        line_list = line.split(" ")
        s = int(float(line_list[5]) * 1000)
        e = s + int(float(line_list[8]) * 1000)
        speaker_ts.append([s, e, int(line_list[11].split("_")[-1])])

wsm = get_words_speaker_mapping(word_timestamps, speaker_ts, "start")

CPU times: user 3.91 ms, sys: 934 µs, total: 4.84 ms
Wall time: 4.71 ms


In [ ]:
print(wsm)

[{'word': 'Buen', 'start_time': 1450, 'end_time': 1590, 'speaker': 0}, {'word': 'día,', 'start_time': 1630, 'end_time': 1730, 'speaker': 0}, {'word': 'le', 'start_time': 1790, 'end_time': 1890, 'speaker': 0}, {'word': 'habla', 'start_time': 1970, 'end_time': 2230, 'speaker': 0}, {'word': 'Vanessa.', 'start_time': 5051, 'end_time': 5191, 'speaker': 1}, {'word': '¿En', 'start_time': 5211, 'end_time': 5291, 'speaker': 1}, {'word': 'qué', 'start_time': 5351, 'end_time': 5471, 'speaker': 1}, {'word': 'le', 'start_time': 5511, 'end_time': 5571, 'speaker': 1}, {'word': 'puedo', 'start_time': 5631, 'end_time': 5851, 'speaker': 1}, {'word': 'ayudar?', 'start_time': 5871, 'end_time': 6191, 'speaker': 1}, {'word': 'Hola,', 'start_time': 6211, 'end_time': 6792, 'speaker': 1}, {'word': 'buenos', 'start_time': 7052, 'end_time': 7232, 'speaker': 1}, {'word': 'días.', 'start_time': 7252, 'end_time': 7552, 'speaker': 1}, {'word': 'Habla', 'start_time': 7572, 'end_time': 7852, 'speaker': 1}, {'word': 'J

## Realligning Speech segments using Punctuation
---

This code provides a method for disambiguating speaker labels in cases where a sentence is split between two different speakers. It uses punctuation markings to determine the dominant speaker for each sentence in the transcription.

```
Speaker A: It's got to come from somewhere else. Yeah, that one's also fun because you know the lows are
Speaker B: going to suck, right? So it's actually it hits you on both sides.
```

For example, if a sentence is split between two speakers, the code takes the mode of speaker labels for each word in the sentence, and uses that speaker label for the whole sentence. This can help to improve the accuracy of speaker diarization, especially in cases where the Whisper model may not take fine utterances like "hmm" and "yeah" into account, but the Diarization Model (Nemo) may include them, leading to inconsistent results.

The code also handles cases where one speaker is giving a monologue while other speakers are making occasional comments in the background. It ignores the comments and assigns the entire monologue to the speaker who is speaking the majority of the time. This provides a robust and reliable method for realigning speech segments to their respective speakers based on punctuation in the transcription.

In [ ]:
%%time
if language in punct_model_langs:
    # restoring punctuation in the transcript to help realign the sentences
    punct_model = PunctuationModel(model="kredor/punctuate-all")

    words_list = list(map(lambda x: x["word"], wsm))

    labled_words = punct_model.predict(words_list)

    ending_puncts = ".?!"
    model_puncts = ".,;:!?"

    # We don't want to punctuate U.S.A. with a period. Right?
    is_acronym = lambda x: re.fullmatch(r"\b(?:[a-zA-Z]\.){2,}", x)

    for word_dict, labeled_tuple in zip(wsm, labled_words):
        word = word_dict["word"]
        if (
            word
            and labeled_tuple[1] in ending_puncts
            and (word[-1] not in model_puncts or is_acronym(word))
        ):
            word += labeled_tuple[1]
            if word.endswith(".."):
                word = word.rstrip(".")
            word_dict["word"] = word

else:
    logging.warning(
        f"Punctuation restoration is not available for {language} language. Using the original punctuation."
    )

wsm = get_realigned_ws_mapping_with_punctuation(wsm)
ssm = get_sentences_speaker_mapping(wsm, speaker_ts)

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

CPU times: user 7.41 s, sys: 4.8 s, total: 12.2 s
Wall time: 20.1 s


In [ ]:
print(ssm)

[{'speaker': 'Speaker 0', 'start_time': 0, 'end_time': 5191, 'text': 'Buen día, le habla Vanessa. '}, {'speaker': 'Speaker 1', 'start_time': 5211, 'end_time': 6191, 'text': '¿En qué le puedo ayudar? '}, {'speaker': 'Speaker 1', 'start_time': 6211, 'end_time': 7552, 'text': 'Hola, buenos días. '}, {'speaker': 'Speaker 1', 'start_time': 7572, 'end_time': 8512, 'text': 'Habla Joan Camilo. '}, {'speaker': 'Speaker 1', 'start_time': 9293, 'end_time': 10613, 'text': 'Perdona la bulla. '}, {'speaker': 'Speaker 1', 'start_time': 11934, 'end_time': 16236, 'text': 'Lo que pasa es que a mí me interesaría ingresar a estudiar enfermería. '}, {'speaker': 'Speaker 0', 'start_time': 18256, 'end_time': 20057, 'text': 'Dime, ya terminaste. '}, {'speaker': 'Speaker 0', 'start_time': 20277, 'end_time': 21978, 'text': 'Dime. '}, {'speaker': 'Speaker 1', 'start_time': 21998, 'end_time': 22258, 'text': 'Perdón. '}, {'speaker': 'Speaker 1', 'start_time': 22691, 'end_time': 37058, 'text': 'Este, me gustaría em

## Cleanup and downloading the results

In [ ]:
%%time
path_textfile_with_speakers = f"{os.path.splitext(audio_path)[0]}.txt"
path_srtfile_with_speakers = f"{os.path.splitext(audio_path)[0]}.srt"

with open(path_textfile_with_speakers, "w", encoding="utf-8-sig") as f:
    get_speaker_aware_transcript(ssm, f)

with open(path_srtfile_with_speakers, "w", encoding="utf-8-sig") as srt:
    write_srt(ssm, srt)

cleanup(temp_path)

CPU times: user 3.6 ms, sys: 2.66 ms, total: 6.26 ms
Wall time: 9.82 ms


In [ ]:
# cleanup text file with speakers
with open(path_textfile_with_speakers, "r") as f:
    lines = f.readlines()
    lines = [re.sub(' +', ' ', line.strip("\ufeff").strip()) for line in lines if line != "\n"]

with open(path_textfile_with_speakers, "w", encoding="utf-8-sig") as f:
    for i,line in enumerate(lines):
        if i < len(lines) - 1: f.write(f"{line}\n\n")
        else: f.write(f"{line}")

### Descargar Archivo

In [ ]:
# download files
from google.colab import files
files.download(path_textfile_with_speakers)
files.download(path_srtfile_with_speakers)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### SETUP BUNCH

## ENLISTA ARCHIVOS

In [ ]:
import os
import glob

# # Listar todos los archivos en la carpeta raíz
# files = os.listdir('/')
# print(files)

# Listar todos los archivos en una carpeta específica, por ejemplo, '/content'
files = os.listdir('/content')
# print(files)

wav_files = [file for file in files if file.endswith('.wav')]
print(wav_files)

# # Usar glob para listar todos los archivos con una extensión específica en una carpeta
# files = glob.glob('/content/*.csv')  # Cambia la extensión según tus necesidades
# print(files)


['a32ef11a-6884-41e7-928b-cf63a386514b_20230506T15_14_UTC.wav', 'ec8aa41a-dfd3-4a1c-b2ea-edf54a679155_20230422T16_33_UTC.wav', '1841240f-f1ab-4df6-96c7-6987b97d4068_20230411T16_35_UTC.wav', '8524c81b-ba37-4acc-a3b5-d1303fab469d_20230603T15_45_UTC.wav', '2423c4d4-01d1-424a-b384-3e5ddda79e7a_20230610T16_50_UTC.wav', 'e0913f30-a778-47f8-9585-f9219bb14dd5_20230610T16_29_UTC.wav', 'ae031a33-d308-40e9-aab5-fd43bf33b425_20230513T15_47_UTC.wav', '1c602ea2-8bef-4b22-9990-ef7ae3bdede8_20230415T14_53_UTC.wav', '5439e31b-4b86-4077-b1e1-4e1fd161a670_20230513T13_55_UTC.wav', '77dc4488-e978-4884-82aa-79d5b5197056_20230411T13_49_UTC.wav', 'f7e4b691-c321-4c6f-b91f-83888f96589f_20230422T14_39_UTC.wav', 'd15a221c-5dce-4a34-ac2d-8bda6ed71ebd_20230610T13_57_UTC.wav', 'ece40026-e149-425b-8c37-f0eb1f33deb4_20230513T15_41_UTC.wav', '2dea44d0-a836-446f-adf6-677088ca215c_20230411T14_37_UTC.wav', '00bcae99-9c09-402c-b74a-20549a188040_20230429T15_03_UTC.wav', 'dee55130-9aee-45f7-981f-694edfbcba81_20230506T15_21_U

In [ ]:
# def rename_file(filepath):
filepath = "00bcae99-9c09-402c-b74a-20549a188040_20230429T15_03_UTC.wav"

filepath = filepath.replace('-','')
suffix = Path(filepath).suffix

# if str(Path(filepath).parent) != ".":
#     new_filepath = str(Path(filepath).parent) + cleanString(filepath.replace(suffix, "")) + suffix
# else:
#     new_filepath = cleanString(filepath.replace(suffix, "")) + suffix
# os.rename(filepath, new_filepath)
# return new_filepath

# print(rename_file("00bcae99-9c09-402c-b74a-20549a188040_20230429T15_03_UTC.wav"))

.


In [ ]:
def isolate_string(audio_path):
  if enable_stemming:
      # Isolate vocals from the rest of the audio

      return_code = os.system(
          f'python3 -m demucs.separate -n htdemucs --two-stems=vocals "{audio_path}" -o "temp_outputs"'
      )

      if return_code != 0:
          logging.warning("Source splitting failed, using original audio file.")
          vocal_target = audio_path
      else:
          vocal_target = os.path.join(
              "temp_outputs",
              "htdemucs",
              os.path.splitext(os.path.basename(audio_path))[0],
              "vocals.wav",
          )
  else:
      vocal_target = audio_path
      return vocal_target

### EJECUCION MUESTRA

In [ ]:
device = "cuda"
vocal_target = audio_path
language = None
whisper_model_name = "large-v3"
# replaces numerical digits with their pronounciation, increases diarization accuracy
suppress_numerals = True
batch_size = 8

if device == "cuda": compute_type = "float16"
# or run on GPU with INT8
# compute_type = "int8_float16"
# or run on CPU with INT8
else: compute_type = "int8"


whisper_results, language = transcribe_batched(
    vocal_target,
    language,
    batch_size,
    whisper_model_name,
    compute_type,
    suppress_numerals,
    device,
)

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:02<00:00, 8.07MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.7. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.
Detected language: es (0.99) in first 30s of audio...
Suppressing numeral and symbol tokens: [3, 4, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 502, 568, 628, 805, 945, 1017, 1025, 1266, 1294, 1360, 1386, 1525, 1614, 1649, 1722, 1848, 1958, 2009, 2119, 2217, 2272, 2319, 2331, 2443, 2625, 2803, 2975, 3165, 3279, 3282, 3356, 3405, 3446, 3499, 3552, 3705, 4022, 4060, 4289, 4303, 4436, 4550, 4688, 4702, 4762, 4808, 5080, 5211, 5254, 5285, 5348, 5853, 5867, 5923, 6071, 6074, 6096, 6375, 6494, 6549, 6591, 6641, 6673, 6856, 6866, 6879, 6905, 6976, 7143, 7201, 7271, 7490, 7526, 7546, 7551, 7560, 7562, 7629, 7634, 7668, 7771, 7773, 7911, 7998, 8132, 8227, 8423, 8451, 8465, 8494, 8652, 8794, 8858, 8923, 9012, 9125, 9356, 9413, 9562, 9657, 9714, 9754, 10076,

### RESULTADO WHISPER

In [ ]:
print(whisper_results)

[{'text': ' Aló. Buenos días, me comunico con Rafael Dávalo. ¿Y con él? Le habla Angie Valente, de la Universidad Javeriana de Cali. ¿Cómo se encuentra? Muy bien, gracias a Dios, usted. Muy bien, gracias a Dios. Nos estamos comunicando, señor Rafael, por su interés en el seminario en Business Analysis con Power BI. Ajá, sí, señora. Bueno, usted se encuentra en la ciudad de Cali.', 'start': 0.623, 'end': 29.599}, {'text': ' Sí, señora. Este seminario se dictará de manera presencial en el campus universitario. Iniciaría el veintidós de junio y finalizaría el treinta y uno de julio. ¿Qué tenés a la pregunta? ¿Este es el curso de Python? No, señor. Seminario en Business y análisis de Big Data con Power BI. Ah, ok, ok. Sí, sí, ya recordé. Este seminario es gratuito, ¿cierto?', 'start': 30.503, 'end': 58.729}, {'text': ' Se estaba, digamos que se estaba ofertando de esa manera, pero pues con un convenio que tenía el ICT con el Ministerio de las TIC, pero esa convocatoria ya finalizó. Ahorita

## ALINEACION DE AUDIO CON WAV2VEC2

In [ ]:
%%time
if language in wav2vec2_langs:
    device = "cuda"
    alignment_model, metadata = whisperx.load_align_model(
        language_code=language, device=device
    )
    result_aligned = whisperx.align(
        whisper_results, alignment_model, metadata, vocal_target, device
    )
    word_timestamps = filter_missing_timestamps(result_aligned["word_segments"])

    # clear gpu vram
    del alignment_model
    torch.cuda.empty_cache()
else:
    assert batch_size == 0, (  # TODO: add a better check for word timestamps existence
        f"Unsupported language: {language}, use --batch_size to 0"
        " to generate word timestamps using whisper directly and fix this error."
    )
    word_timestamps = []
    for segment in whisper_results:
        for word in segment["words"]:
            word_timestamps.append({"word": word[2], "start": word[0], "end": word[1]})

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_voxpopuli_base_10k_asr_es.pt" to /root/.cache/torch/hub/checkpoints/wav2vec2_voxpopuli_base_10k_asr_es.pt
100%|██████████| 360M/360M [00:05<00:00, 66.4MB/s]


CPU times: user 5.36 s, sys: 1.35 s, total: 6.71 s
Wall time: 12.2 s


In [ ]:
print(word_timestamps)

[{'word': 'Aló.', 'start': 0.663, 'end': 0.803, 'score': 0.035}, {'word': 'Buenos', 'start': 0.823, 'end': 3.324, 'score': 0.496}, {'word': 'días,', 'start': 3.385, 'end': 3.725, 'score': 0.698}, {'word': 'me', 'start': 3.745, 'end': 3.825, 'score': 0.544}, {'word': 'comunico', 'start': 3.865, 'end': 4.265, 'score': 0.692}, {'word': 'con', 'start': 4.305, 'end': 4.425, 'score': 0.661}, {'word': 'Rafael', 'start': 4.485, 'end': 4.905, 'score': 0.702}, {'word': 'Dávalo.', 'start': 4.965, 'end': 5.326, 'score': 0.774}, {'word': '¿Y', 'start': 6.486, 'end': 6.546, 'score': 0.568}, {'word': 'con', 'start': 6.586, 'end': 6.686, 'score': 0.436}, {'word': 'él?', 'start': 6.766, 'end': 6.906, 'score': 0.495}, {'word': 'Le', 'start': 7.827, 'end': 7.927, 'score': 0.788}, {'word': 'habla', 'start': 7.987, 'end': 8.147, 'score': 0.447}, {'word': 'Angie', 'start': 8.187, 'end': 8.467, 'score': 0.749}, {'word': 'Valente,', 'start': 8.487, 'end': 8.868, 'score': 0.566}, {'word': 'de', 'start': 8.888,

## CONVERSION A MONO

In [ ]:
%%time
sound = AudioSegment.from_file(vocal_target).set_channels(1)
ROOT = os.getcwd()
temp_path = os.path.join(ROOT, "temp_outputs")
os.makedirs(temp_path, exist_ok=True)
sound.export(os.path.join(temp_path, "mono_file.wav"), format="wav")

CPU times: user 9.98 ms, sys: 5.04 ms, total: 15 ms
Wall time: 15.2 ms


<_io.BufferedRandom name='/content/temp_outputs/mono_file.wav'>

## DIARIZACION AUDIO

In [ ]:
%%time
# Initialize NeMo MSDD diarization model
# DOMAIN_TYPE: can be meeting, telephonic, or general based on domain type of the audio file
msdd_model = NeuralDiarizer(cfg=create_config(temp_path, DOMAIN_TYPE="telephonic")).to("cuda")
msdd_model.diarize()

del msdd_model
torch.cuda.empty_cache()

[NeMo I 2024-08-18 02:42:44 msdd_models:1092] Loading pretrained diar_msdd_telephonic model from NGC
[NeMo I 2024-08-18 02:42:44 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/diar_msdd_telephonic/versions/1.0.1/files/diar_msdd_telephonic.nemo to /root/.cache/torch/NeMo/NeMo_1.21.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo
[NeMo I 2024-08-18 02:42:46 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2024-08-18 02:42:47 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: true
    
[NeMo W 2024-08-18 02:42:47 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: false
    
[NeMo W 2024-08-18 02:42:47 modelPT:174] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2024-08-18 02:42:47 features:289] PADDING: 16
[NeMo I 2024-08-18 02:42:47 features:289] PADDING: 16
[NeMo I 2024-08-18 02:42:48 save_restore_connector:249] Model EncDecDiarLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2024-08-18 02:42:48 features:289] PADDING: 16
[NeMo I 2024-08-18 02:42:49 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2024-08-18 02:42:49 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/vad_multilingual_marblenet/versions/1.10.0/files/vad_multilingual_marblenet.nemo to /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2024-08-18 02:42:49 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2024-08-18 02:42:49 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2024-08-18 02:42:49 features:289] PADDING: 16
[NeMo I 2024-08-18 02:42:49 save_restore_connector:249] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2024-08-18 02:42:50 msdd_models:864] Multiscale Weights: [1, 1, 1, 1, 1]
[NeMo I 2024-08-18 02:42:50 msdd_models:865] Clustering Parameters: {
        "oracle_num_speakers": false,
        "max_num_speakers": 8,
        "enhanced_count_thres": 80,
        "max_rp_threshold": 0.25,
        "sparse_search_volume": 30,
        "maj_vote_spk_count": false,
        "chunk_cluster_count": 50,
        "embeddings_per_chunk": 10000
    }
[NeMo I 2024-08-18 02:42:50 speaker_utils:93] Number of files to diarize: 1
[NeMo I 2024-08-18 02:42:50 clustering_diarizer:309] Split long audio file to avoid CUDA memory issue


...

[NeMo I 2024-08-18 02:43:13 speaker_utils:93] Number of files to diarize: 1


[NeMo W 2024-08-18 02:43:13 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2024-08-18 02:43:13 speaker_utils:93] Number of files to diarize: 1


[NeMo W 2024-08-18 02:43:13 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2024-08-18 02:43:13 msdd_models:1431]   
    
CPU times: user 22.8 s, sys: 930 ms, total: 23.8 s
Wall time: 28.9 s


## MAPEO HABLANTES POR TIEMPO

In [ ]:
%%time
# Reading timestamps <> Speaker Labels mapping

speaker_ts = []
with open(os.path.join(temp_path, "pred_rttms", "mono_file.rttm"), "r") as f:
    lines = f.readlines()
    for line in lines:
        line_list = line.split(" ")
        s = int(float(line_list[5]) * 1000)
        e = s + int(float(line_list[8]) * 1000)
        speaker_ts.append([s, e, int(line_list[11].split("_")[-1])])

wsm = get_words_speaker_mapping(word_timestamps, speaker_ts, "start")

CPU times: user 2.47 ms, sys: 0 ns, total: 2.47 ms
Wall time: 2.29 ms


In [ ]:
print(wsm)

[{'word': 'Aló.', 'start_time': 663, 'end_time': 803, 'speaker': 0}, {'word': 'Buenos', 'start_time': 823, 'end_time': 3324, 'speaker': 0}, {'word': 'días,', 'start_time': 3385, 'end_time': 3725, 'speaker': 1}, {'word': 'me', 'start_time': 3745, 'end_time': 3825, 'speaker': 1}, {'word': 'comunico', 'start_time': 3865, 'end_time': 4265, 'speaker': 1}, {'word': 'con', 'start_time': 4305, 'end_time': 4425, 'speaker': 1}, {'word': 'Rafael', 'start_time': 4485, 'end_time': 4905, 'speaker': 1}, {'word': 'Dávalo.', 'start_time': 4965, 'end_time': 5326, 'speaker': 1}, {'word': '¿Y', 'start_time': 6486, 'end_time': 6546, 'speaker': 0}, {'word': 'con', 'start_time': 6586, 'end_time': 6686, 'speaker': 0}, {'word': 'él?', 'start_time': 6766, 'end_time': 6906, 'speaker': 0}, {'word': 'Le', 'start_time': 7827, 'end_time': 7927, 'speaker': 1}, {'word': 'habla', 'start_time': 7987, 'end_time': 8147, 'speaker': 1}, {'word': 'Angie', 'start_time': 8186, 'end_time': 8467, 'speaker': 1}, {'word': 'Valente

## ESTRUCTURA HABLANTES

In [ ]:
%%time
if language in punct_model_langs:
    # restoring punctuation in the transcript to help realign the sentences
    punct_model = PunctuationModel(model="kredor/punctuate-all")

    words_list = list(map(lambda x: x["word"], wsm))

    labled_words = punct_model.predict(words_list)

    ending_puncts = ".?!"
    model_puncts = ".,;:!?"

    # We don't want to punctuate U.S.A. with a period. Right?
    is_acronym = lambda x: re.fullmatch(r"\b(?:[a-zA-Z]\.){2,}", x)

    for word_dict, labeled_tuple in zip(wsm, labled_words):
        word = word_dict["word"]
        if (
            word
            and labeled_tuple[1] in ending_puncts
            and (word[-1] not in model_puncts or is_acronym(word))
        ):
            word += labeled_tuple[1]
            if word.endswith(".."):
                word = word.rstrip(".")
            word_dict["word"] = word

else:
    logging.warning(
        f"Punctuation restoration is not available for {language} language. Using the original punctuation."
    )

wsm = get_realigned_ws_mapping_with_punctuation(wsm)
ssm = get_sentences_speaker_mapping(wsm, speaker_ts)

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

CPU times: user 5.44 s, sys: 3.8 s, total: 9.25 s
Wall time: 32.3 s


In [ ]:
print(ssm)

[{'speaker': 'Speaker 0', 'start_time': 0, 'end_time': 803, 'text': 'Aló. '}, {'speaker': 'Speaker 1', 'start_time': 823, 'end_time': 5326, 'text': 'Buenos días, me comunico con Rafael Dávalo. '}, {'speaker': 'Speaker 0', 'start_time': 6486, 'end_time': 6906, 'text': '¿Y con él? '}, {'speaker': 'Speaker 1', 'start_time': 7827, 'end_time': 10508, 'text': 'Le habla Angie Valente, de la Universidad Javeriana de Cali. '}, {'speaker': 'Speaker 1', 'start_time': 10548, 'end_time': 11369, 'text': '¿Cómo se encuentra? '}, {'speaker': 'Speaker 0', 'start_time': 12410, 'end_time': 13670, 'text': 'Muy bien, gracias a Dios, usted. '}, {'speaker': 'Speaker 1', 'start_time': 14911, 'end_time': 16052, 'text': 'Muy bien, gracias a Dios. '}, {'speaker': 'Speaker 1', 'start_time': 16932, 'end_time': 23336, 'text': 'Nos estamos comunicando, señor Rafael, por su interés en el seminario en Business Analysis con Power BI. '}, {'speaker': 'Speaker 0', 'start_time': 24796, 'end_time': 25657, 'text': 'Ajá, sí,

## EXPORTACION DE RESULTADOS

In [ ]:
%%time
path_textfile_with_speakers = f"{os.path.splitext(audio_path)[0]}.txt"
path_srtfile_with_speakers = f"{os.path.splitext(audio_path)[0]}.srt"

with open(path_textfile_with_speakers, "w", encoding="utf-8-sig") as f:
    get_speaker_aware_transcript(ssm, f)

with open(path_srtfile_with_speakers, "w", encoding="utf-8-sig") as srt:
    write_srt(ssm, srt)

cleanup(temp_path)

CPU times: user 3.91 ms, sys: 1.03 ms, total: 4.94 ms
Wall time: 10.8 ms


In [ ]:
def create_text(audio_path):
    # rename audio filename if necessary to get string without accent, space, in lower case
    audio_path = rename_file(audio_path)
    # vocal_target = isolate_string(audio_path)
    # whisper_results, language = transcribe(vocal_target)
    return audio_path

# DESARROLLO FOLDER

### ESTANDARIZACION DE FILEPATH

In [ ]:
# Listar todos los archivos en una carpeta específica, por ejemplo, '/content'
files = os.listdir('/content')
# print(files)

wav_files = [file for file in files if file.endswith('.wav')]
# print(wav_files)

for filepath in wav_files:
  suffix = Path(filepath).suffix

  if str(Path(filepath).parent) != ".":
      new_filepath = str(Path(filepath).parent) + cleanString(filepath.replace(suffix, "")) + suffix
  else:
      new_filepath = cleanString(filepath.replace(suffix, "")) + suffix
  os.rename(filepath, new_filepath)


In [ ]:
files = os.listdir('/content')
wav_files = [fl for fl in files if Path(fl).suffix == ".wav"]
print(wav_files)

['b3067024c98b48888d1b93992535ae9a_20230610t14_15_utc.wav', '18b2b93c2953487c97c8b73ce36135fb_20230610t15_48_utc.wav', '033864c87c5f4a4fab44a201563fd968_20230610t15_05_utc.wav', '3030567b3ee042bb8a838d1c76b6b916_20230610t14_01_utc.wav', '7561a59a1cc24d3f9dcb726bda6b777e_20230610t14_18_utc.wav', '2423c4d401d1424ab3843e5ddda79e7a_20230610t16_50_utc.wav', '975df3115269486098f522a3eb856dfd_20230610t15_05_utc.wav', '34958b78223b449782e44c3afd0d0fd7_20230610t14_27_utc.wav', 'd32ed68f007043a4a4eb582c82015c33_20230610t14_59_utc.wav', '3fde7eca0a2f4b188a3fd81fce9a9d09_20230610t14_48_utc.wav', 'ca04630280544cbf8b9bee6854a2f681_20230610t14_13_utc.wav', '5fab44f212e3402d99638a5dc9d97701_20230610t14_26_utc.wav', 'e091c45596054f6ea5fac6b7388e115c_20230610t15_13_utc.wav', 'd9553adbba9c45bead6e1a2bded61ad8_20230610t16_33_utc.wav', '934de5177dd94afd8f8a198b2226c2e5_20230610t15_00_utc.wav', '45adf080f3e24793900699ada4592d63_20230610t15_47_utc.wav', 'eda880c7ca6f4e40835aa372d01627c0_20230610t14_59_utc.wa

In [ ]:
len(wav_files)

59

In [ ]:
#Limpieza de storage
files = os.listdir('/content')
not_txt = [fl for fl in files if Path(fl).suffix != ".txt"]
elm = [fl for fl in files if Path(fl).suffix in (".txt", ".wav")]
# len(elm)
for nt in elm:
  mono_file_path = os.path.join('/content', nt)
  # print(mono_file_path)
  cleanup(mono_file_path)

### PROCESO TRANSCRIPCION

In [ ]:
# wav_files = ['0c1dacc20d21474f9f300b2bc3069d1a_20230401t16_11_utc.wav']

# vocal_target = audio_path
language = None
batch_size = 8
whisper_model_name = "large-v3"
compute_type = "float16"
suppress_numerals = True
device = "cuda"

for vocal_target in wav_files:

  whisper_results, language = transcribe_batched(
      vocal_target,
      language,
      batch_size,
      whisper_model_name,
      compute_type,
      suppress_numerals,
      device,
  )

######################################
#########      WAV2VEC2      #########
######################################

  alignment_model, metadata = whisperx.load_align_model(
      language_code=language, device=device
  )
  result_aligned = whisperx.align(
      whisper_results, alignment_model, metadata, vocal_target, device
  )
  word_timestamps = filter_missing_timestamps(result_aligned["word_segments"])

  # clear gpu vram
  del alignment_model
  torch.cuda.empty_cache()

######################################
######  CONVERSION A MONO   ##########
######################################

  sound = AudioSegment.from_file(vocal_target).set_channels(1)
  ROOT = os.getcwd()
  temp_path = os.path.join(ROOT, "temp_outputs")
  os.makedirs(temp_path, exist_ok=True)
  sound.export(os.path.join(temp_path, "mono_file.wav"), format="wav")

#######################################
########  DIARIZACION AUDIO   #########
#######################################

  msdd_model = NeuralDiarizer(cfg=create_config(temp_path, DOMAIN_TYPE="telephonic")).to("cuda")
  msdd_model.diarize()

  del msdd_model
  torch.cuda.empty_cache()

#######################################
####  MAPEO HABLANTES POR TIEMPO   ####
#######################################


  speaker_ts = []
  with open(os.path.join(temp_path, "pred_rttms", "mono_file.rttm"), "r") as f:
      lines = f.readlines()
      for line in lines:
          line_list = line.split(" ")
          s = int(float(line_list[5]) * 1000)
          e = s + int(float(line_list[8]) * 1000)
          speaker_ts.append([s, e, int(line_list[11].split("_")[-1])])

  wsm = get_words_speaker_mapping(word_timestamps, speaker_ts, "start")

#######################################
#######  ESTRUCTURA HABLANTES   #######
#######################################

  punct_model = PunctuationModel(model="kredor/punctuate-all")

  words_list = list(map(lambda x: x["word"], wsm))

  labled_words = punct_model.predict(words_list)

  ending_puncts = ".?!"
  model_puncts = ".,;:!?"

  # We don't want to punctuate U.S.A. with a period. Right?
  is_acronym = lambda x: re.fullmatch(r"\b(?:[a-zA-Z]\.){2,}", x)

  for word_dict, labeled_tuple in zip(wsm, labled_words):
      word = word_dict["word"]
      if (
          word
          and labeled_tuple[1] in ending_puncts
          and (word[-1] not in model_puncts or is_acronym(word))
      ):
          word += labeled_tuple[1]
          if word.endswith(".."):
              word = word.rstrip(".")
          word_dict["word"] = word

  wsm = get_realigned_ws_mapping_with_punctuation(wsm)
  ssm = get_sentences_speaker_mapping(wsm, speaker_ts)


#######################################
######  EXPORTACION RESULTADOS   ######
#######################################

  path_textfile_with_speakers = f"/content/drive/MyDrive/audios/2023-06-10a/{os.path.splitext(vocal_target)[0]}.txt"  #JSON #Pandas(Speaker 1 - Speaker 2))
  # path_srtfile_with_speakers = f"/content/drive/MyDrive/audios/{os.path.splitext(vocal_target)[0]}.srt"

  with open(path_textfile_with_speakers, "w", encoding="utf-8-sig") as f:
      get_speaker_aware_transcript(ssm, f)

  # with open(path_srtfile_with_speakers, "w", encoding="utf-8-sig") as srt:
  #     write_srt(ssm, srt)

  mono_file_path = os.path.join(temp_path, "mono_file.wav")
  cleanup(mono_file_path)

  print(f'archivo {vocal_target} listo')
# print(whisper_results)
# print(word_timestamps)
# print(wsm)
# print(ssm)

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.7. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.
Detected language: es (0.98) in first 30s of audio...
Suppressing numeral and symbol tokens: [3, 4, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 502, 568, 628, 805, 945, 1017, 1025, 1266, 1294, 1360, 1386, 1525, 1614, 1649, 1722, 1848, 1958, 2009, 2119, 2217, 2272, 2319, 2331, 2443, 2625, 2803, 2975, 3165, 3279, 3282, 3356, 3405, 3446, 3499, 3552, 3705, 4022, 4060, 4289, 4303, 4436, 4550, 4688, 4702, 4762, 4808, 5080, 5211, 5254, 5285, 5348, 5853, 5867, 5923, 6071, 6074, 6096, 6375, 6494, 6549, 6591, 6641, 6673, 6856, 6866, 6879, 6905, 6976, 7143, 7201, 7271, 7490, 7526, 7546, 7551, 7560, 7562, 7629, 7634, 7668, 7771, 7773, 7911, 7998, 8132, 82

[NeMo W 2024-08-26 03:36:11 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: true
    
[NeMo W 2024-08-26 03:36:11 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: false
    
[NeMo W 2024-08-26 03:36:11 modelPT:174] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2024-08-26 03:36:11 features:289] PADDING: 16
[NeMo I 2024-08-26 03:36:12 features:289] PADDING: 16
[NeMo I 2024-08-26 03:36:12 save_restore_connector:249] Model EncDecDiarLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2024-08-26 03:36:13 features:289] PADDING: 16
[NeMo I 2024-08-26 03:36:14 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2024-08-26 03:36:14 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2024-08-26 03:36:14 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2024-08-26 03:36:14 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2024-08-26 03:36:14 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

...

[NeMo I 2024-08-26 04:29:03 speaker_utils:93] Number of files to diarize: 1


[NeMo W 2024-08-26 04:29:03 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2024-08-26 04:29:03 speaker_utils:93] Number of files to diarize: 1


[NeMo W 2024-08-26 04:29:03 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2024-08-26 04:29:03 msdd_models:1431]   
    
archivo 2debb7328d994ecdb89a0a7afd03cb2e_20230610t16_57_utc.wav listo


In [ ]:
#Limpieza Cache
import gc
gc.collect()


30686